In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta

# jax
import jax.numpy as jnp
import time

import sys
import os
import importlib
import pickle
import itertools

path_ = os.path.abspath(os.path.join('..', '00_utils'))
if path_ not in sys.path:
    sys.path.append(path_)

path_ = os.path.abspath(os.path.join('..', '00_models'))
if path_ not in sys.path:
    sys.path.append(path_)

path_ = os.path.abspath(os.path.join('..', '00_utils_training'))
if path_ not in sys.path:
    sys.path.append(path_)

import run_train_pyomo_rl

In [38]:
START_DATE = '2015-01-05'
start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
date_sequences = [start_date + timedelta(days=i*2) for i in range(5)]
date_sequences_str = [date.strftime('%Y-%m-%d') for date in date_sequences]
print(date_sequences_str)

times_elapsed = []

['2015-01-05', '2015-01-07', '2015-01-09', '2015-01-11', '2015-01-13']


In [19]:
experiment_results.keys()

dict_keys(['result', 'times_elapsed', 'mae_odeint', 'mse_odeint', 'mae_coll_ode', 'mse_coll_ode', 'mae_odeint_test', 'mse_odeint_test', 'mae_coll_ode_test', 'mse_coll_ode_test'])

In [21]:
def initialize_metrics():
    # Initialize lists to store metrics for each iteration
    metrics = {
        'times_elapsed': [],
        'mse_odeint': [],
        'mse_coll_ode': [],
        'mse_odeint_test': [],
        'mse_coll_ode_test': []
    }
    return metrics

def collect_metrics(metrics, experiment_results):
    # Extract metrics from the results dictionary and append to the respective lists
    metrics['times_elapsed'].append(experiment_results['times_elapsed'])
    metrics['mse_odeint'].append(experiment_results['mse_odeint'])
    metrics['mse_coll_ode'].append(experiment_results['mse_coll_ode'])
    metrics['mse_odeint_test'].append(experiment_results['mse_odeint_test'])
    metrics['mse_coll_ode_test'].append(experiment_results['mse_coll_ode_test'])

def compute_averages(metrics):
    # Compute average of each metric and return as a new dictionary
    averages = {key: sum(values) / len(values) for key, values in metrics.items()}
    return averages

* [X] Regularization Strength with Fixed Tolerance
* [] 'tol' with no regularization
* [] 

In [47]:
# aim: loop accross different penalties and see how the results change
# average results over different starting dates

LS = [7, 32, 1]
PENALTY = 0
RESULTS_FULL= {}
RESULTS_AVG = {}

importlib.reload(run_train_pyomo_rl)

params_data = {'file_path': '../00_data/df_train.csv', 'start_date': START_DATE, 
            'n_points': 400, 'split': 200, 'n_days': 1, 'm': 1, 
            'encoding': {'settlement_date': 't', 'temperature': 'var1', 'hour': 'var2', 'nd': 'y'}}

params_solver = {
    "tol" : 1e-5, 
    #"dual_inf_tol": 1e-4, 
    "compl_inf_tol": 1e-3, 
    "constr_viol_tol": 1e-3, 
    "acceptable_constr_viol_tol": 1e-15,
    "acceptable_dual_inf_tol": 1e-15,
    "acceptable_compl_inf_tol": 1e-15,
    "halt_on_ampl_error" : 'yes', "print_level": 1, "max_iter": 500,
    'warm_start_init_point': 'yes'}

penalty_values = [0, 1e-7, 1e-5, 1e-3, 0.01, 0.1]
tol_list = [1, 1e-1, 1e-2, 1e-4, 1e-6, 1e-8]

param_combinations = list(itertools.product(penalty_values, tol_list))
with open('results.txt', 'w'):
    pass
file = open('results.txt', 'a')

for param_comb in param_combinations:
    # ------------------------------------------ UPDATE PARAMETERS ------------------------------------------
    PENALTY = param_comb[0]
    params_solver['tol'] = param_comb[1]
    params_solver['constr_viol_tol'] = param_comb[1]
    #params_solver['compl_inf_tol'] = param_comb
    #params_solver['constr_viol_tol'] = param_comb
    metrics = initialize_metrics()
    
    for i, date in enumerate(date_sequences):
        print(f"param_comb: {param_comb}, date: {date}")
        
        # ------------------------------------------ UPDATE DATE -------------------------------------------
        params_data['start_date'] = date
        
        Trainer = run_train_pyomo_rl.Trainer
        params_model = {'layer_sizes': LS, 'penalty': PENALTY}
        trainer = Trainer(params_data, params_model, params_solver)
        experiment_results = trainer.train()
        
        RESULTS_FULL[(param_comb, date)] = experiment_results
        
        collect_metrics(metrics, experiment_results)
        print(f"param_comb: {param_comb}, date: {date}, results: {experiment_results}")
        
        file.write(f"param_comb: {param_comb}, date: {date}, results: {experiment_results}\n")
        file.flush() 

    RESULTS_AVG[param_comb] = compute_averages(metrics)
    

param_comb: 1, date: 2015-01-05 00:00:00
current_14_08
y
Ipopt 3.14.16: tol=1
compl_inf_tol=0.001
constr_viol_tol=0.001
acceptable_constr_viol_tol=1e-15
acceptable_dual_inf_tol=1e-15
acceptable_compl_inf_tol=1e-15
halt_on_ampl_error=yes
print_level=1
max_iter=500
warm_start_init_point=yes


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

Ipopt 3.14.16: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt

In [35]:
reload = True
if reload:
    formatted_time = time.strftime('%Y-%m-%d_%H-%M-%S')
    filename = f'results/{formatted_time}_tol_full.pkl'
    with open(filename, 'wb') as file:
        pickle.dump(RESULTS_FULL, file)

In [31]:
RESULTS_FULL

{(0,
  '2015-01-05'): {'result': {'solver_time': 164.43716096878052,
   'termination_condition': <TerminationCondition.maxIterations: 'maxIterations'>,
   'message': 'Ipopt 3.14.16\\x3a Maximum Number of Iterations Exceeded.'}, 'times_elapsed': 164.43716096878052, 'mse_odeint': 1.8516538138729286, 'mse_coll_ode': 0.13517974751774087, 'mse_odeint_test': 6.7347220614451535, 'mse_coll_ode_test': 0.5683386185523489},
 (0,
  '2015-01-06'): {'result': {'solver_time': 131.68245387077332,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
   'message': 'Ipopt 3.14.16\\x3a Optimal Solution Found'}, 'times_elapsed': 131.68245387077332, 'mse_odeint': 9.340066723841849, 'mse_coll_ode': 0.23896215132140255, 'mse_odeint_test': 4.800421697259416, 'mse_coll_ode_test': 0.28534492235908826},
 (0,
  '2015-01-07'): {'result': {'solver_time': 107.63502407073975,
   'termination_condition': <TerminationCondition.optimal: 'optimal'>,
   'message': 'Ipopt 3.14.16\\x3a Optimal Solution Foun

In [30]:
RESULTS_AVG

{0: {'times_elapsed': 139.41963868141175,
  'mse_odeint': 19.88011375705098,
  'mse_coll_ode': 0.10626959554879614,
  'mse_odeint_test': 61.317962800964914,
  'mse_coll_ode_test': 1.8841772441584088},
 1e-07: {'times_elapsed': 130.6343058347702,
  'mse_odeint': 70.07915764851563,
  'mse_coll_ode': 0.12358719819233772,
  'mse_odeint_test': 105.4781464025961,
  'mse_coll_ode_test': 3.4540188149787765},
 1e-05: {'times_elapsed': 30.514441299438477,
  'mse_odeint': 625.1751390607817,
  'mse_coll_ode': 0.07638442090908407,
  'mse_odeint_test': 824.7603536801889,
  'mse_coll_ode_test': 0.6116054500791219},
 0.001: {'times_elapsed': 10.904995536804199,
  'mse_odeint': 251.57860901573858,
  'mse_coll_ode': 1.029935861763332,
  'mse_odeint_test': 528.7164708980087,
  'mse_coll_ode_test': 2.2159159499715497},
 0.01: {'times_elapsed': 5.635345768928528,
  'mse_odeint': 0.7853463310603946,
  'mse_coll_ode': 0.8078977253203632,
  'mse_odeint_test': 3.206010632499814,
  'mse_coll_ode_test': 3.417864